In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
from nltk.corpus import stopwords
import re

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
df = pd.read_csv("mbti_1_preprocessedv2.csv")

tweets = df.posts.values
labels = df.type.values

print(tweets[0])
print(len(labels))

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
max_len = 0

for sent in tweets:
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    max_len = max(max_len, len(input_ids))

print(max_len)


cuda:0
', , enfp and intj moments sportscenter not top ten plays pranks, what has been the most life-changing experience in your life?, on repeat for most of today., may the perc experience immerse you., the last thing my infj friend posted on his facebook before committing suicide the next day. rest in peace~ , hello enfj7. sorry to hear of your distress. it's only natural for a relationship to not be perfection all the time in every moment of existence. try to figure the hard times as times of growth, as..., 84389 84390 ..., welcome and stuff., game. set. match., prozac, wellbrutin, at least thirty minutes of moving your legs (and i don't mean moving them while sitting in your same desk chair), weed in moderation (maybe try edibles as a healthier alternative..., basically come up with three items you've determined that each type (or whichever types you want to
8675


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

404


In [ ]:
import nltk
from nltk.corpus import stopwords
import torch
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, random_split
import pandas as pd
import numpy as np
from imblearn.over_sampling import RandomOverSampler


nltk.download('stopwords')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 데이터 로드
df = pd.read_csv("mbti_1_preprocessedv2.csv")
tweets = df.posts.values
labels = df.type.values

# BERT tokenizer 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
max_len = 0
for sent in tweets:
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    max_len = max(max_len, len(input_ids))

# 토큰 및 어텐션 마스크
input_ids = []
attention_masks = []
for tweet in tweets:
    encoded_dict = tokenizer.encode_plus(
                        tweet,
                        add_special_tokens=True,
                        max_length=max_len,
                        pad_to_max_length=True,
                        return_attention_mask=True,
                        return_tensors='pt'
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

# 텐서 변환
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

label_values = [
    'INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP',
    'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ'
]

label_mapping = {label: int(label[3] == 'J') for label in label_values}

# numpy 배열 변환
labels = np.array([label_mapping[label] for label in labels])
print("\nLabel mapping:\n", label_mapping)

# 레이블 텐서 변환
labels = torch.tensor(labels)
dataset = TensorDataset(input_ids, attention_masks, labels)


#####데이터셋 비율이 너무 안맞아서 정확도가 너무 떨어짐, 오버샘플링을 통해서 비율 맞춰줌
# 데이터셋 오버샘플링
oversample = RandomOverSampler(sampling_strategy='minority')
X_over, y_over = oversample.fit_resample(input_ids.numpy(), labels.numpy())

input_ids_over = torch.tensor(X_over)
labels_over = torch.tensor(y_over)

attention_masks_over = torch.ones_like(input_ids_over)
oversampled_dataset = TensorDataset(input_ids_over, attention_masks_over, labels_over)

# 데이터 분할
train_size = int(0.8 * len(oversampled_dataset))
val_size = len(oversampled_dataset) - train_size
train_dataset, val_dataset = random_split(oversampled_dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  war


Label mapping:
 {'INFJ': 1, 'ENTP': 0, 'INTP': 0, 'INTJ': 1, 'ENTJ': 1, 'ENFJ': 1, 'INFP': 0, 'ENFP': 0, 'ISFP': 0, 'ISTP': 0, 'ISFJ': 1, 'ISTJ': 1, 'ESTP': 0, 'ESFP': 0, 'ESTJ': 1, 'ESFJ': 1}
8,385 training samples
2,097 validation samples


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

input_ids = []
attention_masks = []


for tweet in tweets:
    encoded_dict = tokenizer.encode_plus(
                        tweet,
                        add_special_tokens = True,
                        max_length = max_len,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                   )

    input_ids.append(encoded_dict['input_ids'])

    attention_masks.append(encoded_dict['attention_mask'])
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

label_values = [
    'INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP',
    'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ'
]

label_mapping = {label: int(label[3] == 'J') for label in label_values}

labels = np.array([label_mapping[label] for label in labels])

print("\nLabel mapping:\n", label_mapping)

labels = torch.tensor(labels)

dataset = TensorDataset(input_ids, attention_masks, labels)

train_size = int(0.8 * len(dataset))
val_size = len(dataset)  - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Label mapping:
 {'INFJ': 1, 'ENTP': 0, 'INTP': 0, 'INTJ': 1, 'ENTJ': 1, 'ENFJ': 1, 'INFP': 0, 'ENFP': 0, 'ISFP': 0, 'ISTP': 0, 'ISFJ': 1, 'ISTJ': 1, 'ESTP': 0, 'ESFP': 0, 'ESTJ': 1, 'ESFJ': 1}
6,940 training samples
1,735 validation samples


In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2,
     output_attentions = False, # Whether the model returns attentions weights.
     output_hidden_states = False, # Whether the model returns all hidden-states.
)

model = model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 이전에 저장한 모델 로드
model = torch.load('2nd_bert_EandI_model.pth')

In [ ]:
batch_size = 16

train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size
        )
validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size
        )


optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8
                )

epochs = 10

total_steps = len(train_dataloader) * epochs


#학습률 스케쥴러
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)


def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()


# For each epoch...
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    # Measure how long the training epoch takes.
    t0 = time.time()
    total_train_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        output = model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels)
        loss = output.loss
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
    avg_train_loss = total_train_loss / len(train_dataloader)

  #시간 측정
    training_time = format_time(time.time() - t0)
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))


    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")
    t0 = time.time()
    model.eval()
    total_eval_accuracy = 0
    best_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        with torch.no_grad():
            output= model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
        loss = output.loss
        total_eval_loss += loss.item()
        logits = output.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    validation_time = format_time(time.time() - t0)
    if avg_val_accuracy > best_eval_accuracy:
        torch.save(model, 'oversample_ver_bert_JandP_model.pth')
        best_eval_accuracy = avg_val_accuracy
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 10 ========
Training...

  Average training loss: 0.68
  Training epcoh took: 0:03:38

Running Validation...
  Accuracy: 0.66

======== Epoch 2 / 10 ========
Training...

  Average training loss: 0.61
  Training epcoh took: 0:03:36

Running Validation...
  Accuracy: 0.71

======== Epoch 3 / 10 ========
Training...

  Average training loss: 0.49
  Training epcoh took: 0:03:36

Running Validation...
  Accuracy: 0.74

======== Epoch 4 / 10 ========
Training...

  Average training loss: 0.35
  Training epcoh took: 0:03:37

Running Validation...
  Accuracy: 0.76

======== Epoch 5 / 10 ========
Training...

  Average training loss: 0.23
  Training epcoh took: 0:03:37

Running Validation...
  Accuracy: 0.76

======== Epoch 6 / 10 ========
Training...

  Average training loss: 0.16
  Training epcoh took: 0:03:37

Running Validation...
  Accuracy: 0.77

======== Epoch 7 / 10 ========
Training...

  Average training loss: 0.12
  Training epcoh took: 0:03:37

Running Validatio